
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [1]:
using CSV, DataFrames, Statistics, Dates, Gadfly

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [2]:
data = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [3]:
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [4]:
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [5]:
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data,5)
data

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0
6,0642-01D,2013-05-06,0
7,0642-01D,2013-05-07,0
8,0642-01D,2013-05-08,0
9,0642-01D,2013-05-09,0


#### Exclusion des lignes où :SURVERSE est manquante

In [6]:
surverse_df = dropmissing(data, disallowmissing=true);

In [7]:
n₁ = sum(x->x==1, surverse_df[:SURVERSE], dims=1) 
n₀ = sum(x->x==0, surverse_df[:SURVERSE], dims=1) 
n = n₀ + n₁

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[7]:1
└ @ Core In[7]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[7]:2
└ @ Core In[7]:2


1-element Array{Int64,1}:
 161098

In [8]:
filtervals = ["3260-01D"; "3350-07D"; "4240-01D"; "4350-01D"; "4380-01D"]
surverse_df1 = filter(row-> row.NO_OUVRAGE == filtervals[1], surverse_df)
surverse_df2 = filter(row-> row.NO_OUVRAGE == filtervals[2], surverse_df)
surverse_df3 = filter(row-> row.NO_OUVRAGE == filtervals[3], surverse_df)
surverse_df4 = filter(row-> row.NO_OUVRAGE == filtervals[4], surverse_df)
surverse_df5 = filter(row-> row.NO_OUVRAGE == filtervals[5], surverse_df);

In [9]:
#### on prend pour chaque ouvrage le nombre de fois ou il a eu surver et non
n₁ = Int64[]
n₀  = Int64[]
n  = Int64[]

0-element Array{Int64,1}

In [10]:
n1₁ = sum(x->x==1, surverse_df1[:SURVERSE], dims=1) 
push!(n₁, n1₁[1])
n1₀ = sum(x->x==0, surverse_df1[:SURVERSE], dims=1)  
push!(n₀, n1₀[1])
n1= n1₁[1] + n1₀[1]
push!(n, n1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[10]:1
└ @ Core In[10]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[10]:3
└ @ Core In[10]:3


1-element Array{Int64,1}:
 1097

In [11]:
n1₁ = sum(x->x==1, surverse_df2[:SURVERSE], dims=1) 
push!(n₁, n1₁[1])
n1₀ = sum(x->x==0, surverse_df2[:SURVERSE], dims=1)  
push!(n₀, n1₀[1])
n1= n1₁[1] + n1₀[1]
push!(n, n1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[11]:1
└ @ Core In[11]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[11]:3
└ @ Core In[11]:3


2-element Array{Int64,1}:
 1097
  729

In [12]:
n1₁ = sum(x->x==1, surverse_df3[:SURVERSE], dims=1) 
push!(n₁, n1₁[1])
n1₀ = sum(x->x==0, surverse_df3[:SURVERSE], dims=1)  
push!(n₀, n1₀[1])
n1= n1₁[1] + n1₀[1]
push!(n, n1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[12]:1
└ @ Core In[12]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[12]:3
└ @ Core In[12]:3


3-element Array{Int64,1}:
 1097
  729
 1100

In [13]:
n1₁ = sum(x->x==1, surverse_df4[:SURVERSE], dims=1) 
push!(n₁, n1₁[1])
n1₀ = sum(x->x==0, surverse_df4[:SURVERSE], dims=1)  
push!(n₀, n1₀[1])
n1= n1₁[1] + n1₀[1]
push!(n, n1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[13]:1
└ @ Core In[13]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[13]:3
└ @ Core In[13]:3


4-element Array{Int64,1}:
 1097
  729
 1100
 1100

In [14]:
n1₁ = sum(x->x==1, surverse_df5[:SURVERSE], dims=1) 
push!(n₁, n1₁[1])
n1₀ = sum(x->x==0, surverse_df5[:SURVERSE], dims=1)  
push!(n₀, n1₀[1])
n1= n1₁[1] + n1₀[1]
push!(n, n1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[14]:1
└ @ Core In[14]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[14]:3
└ @ Core In[14]:3


5-element Array{Int64,1}:
 1097
  729
 1100
 1100
 1103

## Chargement des précipitations

In [15]:
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [16]:
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 
data = filter(row -> year(row.date) < 2019, data) ;

# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

In [17]:
pcp_sum = by(data, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
first(pcp_sum ,5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,missing
2,2013-05-02,0,0,0,0,missing
3,2013-05-03,0,0,0,0,missing
4,2013-05-04,0,0,0,0,missing
5,2013-05-05,0,0,0,0,missing


#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [18]:
pcp_max = by(data, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(pcp_max,5)

,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,missing
2,2013-05-02,0,0,0,0,missing
3,2013-05-03,0,0,0,0,missing
4,2013-05-04,0,0,0,0,missing
5,2013-05-05,0,0,0,0,missing


In [25]:
for j=1:size(pcp_sum,1)
    println("$(pcp_sum[j])")
end

Date[2013-05-01, 2013-05-02, 2013-05-03, 2013-05-04, 2013-05-05, 2013-05-06, 2013-05-07, 2013-05-08, 2013-05-09, 2013-05-10, 2013-05-11, 2013-05-12, 2013-05-13, 2013-05-14, 2013-05-15, 2013-05-16, 2013-05-17, 2013-05-18, 2013-05-19, 2013-05-20, 2013-05-21, 2013-05-22, 2013-05-23, 2013-05-24, 2013-05-25, 2013-05-26, 2013-05-27, 2013-05-28, 2013-05-29, 2013-05-30, 2013-05-31, 2013-06-01, 2013-06-02, 2013-06-03, 2013-06-04, 2013-06-05, 2013-06-06, 2013-06-07, 2013-06-08, 2013-06-09, 2013-06-10, 2013-06-11, 2013-06-12, 2013-06-13, 2013-06-14, 2013-06-15, 2013-06-16, 2013-06-17, 2013-06-18, 2013-06-19, 2013-06-20, 2013-06-21, 2013-06-22, 2013-06-23, 2013-06-24, 2013-06-25, 2013-06-26, 2013-06-27, 2013-06-28, 2013-06-29, 2013-06-30, 2013-07-01, 2013-07-02, 2013-07-03, 2013-07-04, 2013-07-05, 2013-07-06, 2013-07-07, 2013-07-08, 2013-07-09, 2013-07-10, 2013-07-11, 2013-07-12, 2013-07-13, 2013-07-14, 2013-07-15, 2013-07-16, 2013-07-17, 2013-07-18, 2013-07-19, 2013-07-20, 2013-07-21, 2013-07-22,

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[25]:2
└ @ Core .\In[25]:2



Union{Missing, Int64}[0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 106, 0, 10, 0, 40, 0, 0, 0, 12, 64, 40, 26, 232, 52, 116, 116, 0, 11, 146, 0, 0, 222, 68, 0, 24, 0, 50, 83, 80, 0, 0, 358, 30, 0, 0, 0, 124, 0, 0, 0, 0, 0, 102, 0, 117, 0, 10, 0, 275, 0, 0, 0, 0, 0, 0, 0, 0, 138, 10, 0, 187, 0, 10, 0, 0, 0, 0, 240, 10, 140, 60, 0, 0, 394, 0, 0, 0, 0, 94, 66, 18, 0, 102, 0, 0, 12, 0, 0, 0, 0, 72, 0, 0, 0, 104, 48, 0, 0, 0, 0, 0, 0, 0, 212, 0, 0, 10, 70, 0, 10, 0, 138, 0, 0, 215, 10, 0, 0, 0, 0, 0, 0, 22, missing, 362, 0, 0, 12, 86, 0, 0, 0, 0, 88, 85, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 206, 10, 0, missing, 0, 0, 0, 24, 0, 12, 114, 58, 0, 60, 0, 16, 0, 0, 0, 38, 10, 28, 0, 0, 202, missing, 0, 14, 211, 11, 0, 0, 0, 22, 68, 0, 0, 0, 0, 0, 152, 300, 0, 0, 0, 0, 44, 0, 0, 22, missing, 86, 10, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, missing, m

Union{Missing, Int64}[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 138, 0, 0, 0, 10, 0, 0, 0, 32, 38, 39, 0, 108, 59, 79, 172, 0, missing, 189, 0, 0, 117, 121, 0, 0, 0, 45, 78, 61, 0, 0, 304, 37, 0, 0, 0, 103, 0, 0, 0, 0, 0, 115, 0, 63, 40, 0, 0, 171, 0, 0, 0, 0, missing, 0, 15, 0, 69, 15, 0, 21, 0, 0, 0, 0, 0, 0, 220, 23, 209, 52, 0, 0, 107, 0, 0, 0, 0, 110, 28, 0, 0, 72, 0, 5, 0, 0, 0, 0, 0, 61, 0, 0, 0, 109, 31, 0, 0, 0, 0, 0, 0, 0, 67, missing, 0, 4, 37, 0, 42, 0, 176, 0, 0, 196, 0, 0, 0, 0, 0, 3, 0, 21, 53, 320, 30, 0, 3, 108, 0, 0, 0, 0, 83, 73, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 53, 128, 0, 0, 0, 0, 0, 0, 28, 0, 7, 113, 45, 16, 46, 0, 13, missing, 0, 0, 25, 3, 15, 0, 0, 183, missing, 0, 5, 179, missing, 0, 0, 0, 6, 68, 0, 0, 3, 0, 0, 195, 192, 0, 0, 0, missing, 2, 0, 0, 38, 30, 39, 2, 0, 15, 0, 0, 0, 209, 0, 0, 0, 0, 0, 0, 0, 15, 319, 393, 24, 0, 0, 150, 90, 0, 0, 0, 0, 0, 435, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 90, 62, 0, 0, 0, 0, 97, 0, 21, 0, 0, 0, 0, 0, 0, 0, 113, 0, 0, 0, 91, 82, 0,

BoundsError: BoundsError: attempt to access String

#### Inclusion dans un dataframe de ces deux variables explicatives potentielles

In [20]:
ouvrage = "4350-01D"
MAXlowerHalf = Float64[]
SUMlowerHalf = Float64[]
for j=1:size(filtervals,1)
    df = filter(row -> row.NO_OUVRAGE == filtervals[j], surverse_df)

    x₁ = Array{Union{Missing, Int64}}(missing, size(df,1)) # variable pour la somme journalière
    x₂ = Array{Union{Missing, Int64}}(missing, size(df,1)) # variable pour le max journalier


    for i=1:size(df,1)

        ind = findfirst(pcp_sum[:,:date] .== df[i,:DATE])

        x₁[i] = pcp_sum[ind,:McTavish]

        ind = findfirst(pcp_max[:,:date] .== df[i,:DATE])

        x₂[i] = pcp_max[ind,:McTavish]

    end

    df[!,:SUM] = x₁
    df[!,:MAX] = x₂

    dropmissing!(df, [:SUM, :MAX],disallowmissing=true)
    dfRP = filter(row -> row.SURVERSE == 1, df)
    qMAx = quantile(dfRP[:MAX], [0.25, 0.5, 0.75])
    qsum = quantile(dfRP[:SUM], [0.25, 0.5, 0.75])
    push!(MAXlowerHalf, qMAx[1])
    push!(SUMlowerHalf, qsum[1])
end
MAXlowerHalf

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[20]:28
└ @ Core In[20]:28
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[20]:29
└ @ Core In[20]:29


5-element Array{Float64,1}:
 37.25
 23.25
 36.5 
 61.0 
 46.0 

In [21]:
for i=1:size(surverse_df1[:SURVERSE], 1)
        ind = findfirst(pcp_sum[:,:date] .== surverse_df1[i,:DATE])
        println("$(pcp_sum[ind,:McTavish])")
end


0
0
0
0
0
0
0
0
10
0
106
0
10
0
40
0
0
0
12
64
40
26
232
52
116
116
0
11
146
0
0
222
68
0
24
0
50
83
80
0
0
358
30
0
0
0
124
0
0
0
0
0
102
0
117
0
10
0
275
0
0
0
0
0
0
0
0
138
10
0
187
0
10
0
0
0
0
240
10
140
60
0
0
394
0
0
0
0
94
66
18
0
102
0
0
12
0
0
0


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[21]:1
└ @ Core .\In[21]:1


0
72
0
0
0
104
48
0
0
0
0
0
0
0
212
0
0
10
70
0
10
0
138
0
0
215
10
0
0
0
0
0
0
22
missing
362
0
0
12
86
0
0
0
0
88
85
0
0
0
0
0
0
0
0
0
0
0
0
0
36
206
10
0
missing
0
0
0
24
0
12
114
58
0
60
0
16
0
0
0
38
10
28
0
0
202
missing
0
14
211
11
0
0
0
22
68
0
0
0
0
0
152
300
0
0
0
0
44
0
0
22
missing
86
10
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
missing
0
4
152
132
2
146
0
0
0
6
14
6
10
0
0
0
0
12
618
0
48
182
12
0
0
0
0
0
0
0
0
0
2
0
0
0
108
0
46
2
0
36
112
0
0
missing
missing
70
0
176
0
0
8
4
10
missing
0
20
0
8
0
0
0
0
0
0
0
0
0
0
284
4
0
54
316
8
0
0
0
0
0
38
86
2
20
missing
missing
34
0


#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que la somme des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [22]:
plot(df, x=:SURVERSE, y=:SUM, Geom.boxplot)

UndefVarError: UndefVarError: df not defined

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que le maximum journalier des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [23]:
plot(df, x=:SURVERSE, y=:MAX, Geom.boxplot)

UndefVarError: UndefVarError: df not defined

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [24]:

# Chargement du fichier de test
test = CSV.read("data/test.csv")

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
n = size(test,1)
surverse = rand(n) .> .5


# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=surverse)
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.
ID

283-element Array{String,1}:
 "3260-01D_2019-05-02"
 "3260-01D_2019-05-09"
 "3260-01D_2019-05-10"
 "3260-01D_2019-05-15"
 "3260-01D_2019-05-20"
 "3260-01D_2019-05-23"
 "3260-01D_2019-05-24"
 "3260-01D_2019-05-26"
 "3260-01D_2019-05-30"
 "3350-07D_2019-05-01"
 "3350-07D_2019-05-02"
 "3350-07D_2019-05-08"
 "3350-07D_2019-05-09"
 ⋮                    
 "4380-01D_2019-09-01"
 "4380-01D_2019-09-02"
 "4380-01D_2019-09-04"
 "4380-01D_2019-09-05"
 "4380-01D_2019-09-12"
 "4380-01D_2019-09-13"
 "4380-01D_2019-09-16"
 "4380-01D_2019-09-22"
 "4380-01D_2019-09-26"
 "4380-01D_2019-09-28"
 "4380-01D_2019-09-29"
 "4380-01D_2019-09-30"